In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Eczema"
cohort = "GSE123088"

# Input paths
in_trait_dir = "../../input/GEO/Eczema"
in_cohort_dir = "../../input/GEO/Eczema/GSE123088"

# Output paths
out_data_file = "../../output/preprocess/Eczema/GSE123088.csv"
out_gene_data_file = "../../output/preprocess/Eczema/gene_data/GSE123088.csv"
out_clinical_data_file = "../../output/preprocess/Eczema/clinical_data/GSE123088.csv"
json_path = "../../output/preprocess/Eczema/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"A validated single-cell-based strategy to identify diagnostic and therapeutic targets in complex diseases"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['cell type: CD4+ T cells'], 1: ['primary diagnosis: ASTHMA', 'primary diagnosis: ATHEROSCLEROSIS', 'primary diagnosis: BREAST_CANCER', 'primary diagnosis: CHRONIC_LYMPHOCYTIC_LEUKEMIA', 'primary diagnosis: CROHN_DISEASE', 'primary diagnosis: ATOPIC_ECZEMA', 'primary diagnosis: HEALTHY_CONTROL', 'primary diagnosis: INFLUENZA', 'primary diagnosis: OBESITY', 'primary diagnosis: PSORIASIS', 'primary diagnosis: SEASONAL_ALLERGIC_RHINITIS', 'primary diagnosis: TYPE_1_DIABETES', 'primary diagnosis: ACUTE_TONSILLITIS', 'primary diagnosis: ULCERATIVE_COLITIS', 'primary diagnosis: Breast cancer', 'primary diagnosis: Control'], 2: ['Sex: Male', 'diagnosis2: ATOPIC_ECZEMA', 'Sex: Female'

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Determine gene expression data availability
# This dataset appears to be a SuperSeries combining several studies
# Since it mentions CD4+ T cells and includes various diagnoses, it likely contains gene expression data
is_gene_available = True

# 2.1 Data Availability
# Trait (Eczema) appears in row 1 as "primary diagnosis: ATOPIC_ECZEMA"
trait_row = 1

# Age appears in row 3 and continues in row 4
age_row = 3

# Gender/Sex appears in rows 2 and 3
gender_row = 2

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    if pd.isna(value):
        return None
    
    # Extract value after colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Check if Eczema is present in any form
    if "ATOPIC_ECZEMA" in value:
        return 1
    elif "HEALTHY_CONTROL" in value or "Control" in value:
        return 0
    else:
        return None

def convert_age(value):
    if pd.isna(value):
        return None
    
    # Extract value after colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    if pd.isna(value):
        return None
    
    # Extract value after colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if value.lower() == "female":
        return 0
    elif value.lower() == "male":
        return 1
    else:
        return None

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (if trait_row is not None)
if trait_row is not None:
    # Extract clinical features
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted features
    preview = preview_df(clinical_df)
    print("Preview of clinical features:")
    print(preview)
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of clinical features:
{'GSM3494884': [nan, 56.0, 1.0], 'GSM3494885': [nan, nan, nan], 'GSM3494886': [nan, 20.0, 0.0], 'GSM3494887': [nan, 51.0, 0.0], 'GSM3494888': [nan, 37.0, 1.0], 'GSM3494889': [nan, 61.0, 1.0], 'GSM3494890': [nan, nan, nan], 'GSM3494891': [nan, 31.0, 1.0], 'GSM3494892': [nan, 56.0, 0.0], 'GSM3494893': [nan, 41.0, 0.0], 'GSM3494894': [nan, 61.0, 0.0], 'GSM3494895': [nan, nan, nan], 'GSM3494896': [nan, 80.0, 1.0], 'GSM3494897': [nan, 53.0, 1.0], 'GSM3494898': [nan, 61.0, 1.0], 'GSM3494899': [nan, 73.0, 1.0], 'GSM3494900': [nan, 60.0, 1.0], 'GSM3494901': [nan, 76.0, 1.0], 'GSM3494902': [nan, 77.0, 0.0], 'GSM3494903': [nan, 74.0, 0.0], 'GSM3494904': [nan, 69.0, 1.0], 'GSM3494905': [nan, 77.0, 0.0], 'GSM3494906': [nan, 81.0, 0.0], 'GSM3494907': [nan, 70.0, 0.0], 'GSM3494908': [nan, 82.0, 0.0], 'GSM3494909': [nan, 69.0, 0.0], 'GSM3494910': [nan, 82.0, 0.0], 'GSM3494911': [nan, 67.0, 0.0], 'GSM3494912': [nan, 67.0, 0.0], 'GSM3494913': [nan, 78.0, 0.0], 'GSM3494914'

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Eczema/GSE123088/GSE123088_series_matrix.txt.gz


Gene data shape: (24166, 204)
First 20 gene/probe identifiers:
Index(['1', '2', '3', '9', '10', '12', '13', '14', '15', '16', '18', '19',
       '20', '21', '22', '23', '24', '25', '26', '27'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers appear to be numeric IDs, not human gene symbols.
# They are likely probe IDs or some other form of identifiers that need to be mapped.
# Looking at the first 20 identifiers, they are simply numbers like '1', '2', '3', etc.
# These are not standard human gene symbols, which would typically be alphanumeric like 'BRCA1', 'TP53', etc.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Let's look for platform information in the SOFT file to understand the annotation better
print("\nSearching for platform information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if '!Series_platform_id' in line:
            print(line.strip())
            break
        if i > 100:  # Limit search to first 100 lines
            print("Platform ID not found in first 100 lines")
            break

# Check if the SOFT file includes any reference to gene symbols
print("\nSearching for gene symbol information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    gene_symbol_lines = []
    for i, line in enumerate(f):
        if 'GENE_SYMBOL' in line or 'gene_symbol' in line.lower() or 'symbol' in line.lower():
            gene_symbol_lines.append(line.strip())
        if i > 1000 and len(gene_symbol_lines) > 0:  # Limit search but ensure we found something
            break
    
    if gene_symbol_lines:
        print("Found references to gene symbols:")
        for line in gene_symbol_lines[:5]:  # Show just first 5 matches
            print(line)
    else:
        print("No explicit gene symbol references found in first 1000 lines")

# Look for alternative annotation files or references in the directory
print("\nChecking for additional annotation files in the directory:")
all_files = os.listdir(in_cohort_dir)
print([f for f in all_files if 'annotation' in f.lower() or 'platform' in f.lower() or 'gpl' in f.lower()])



Gene annotation preview:
Columns in gene annotation: ['ID', 'ENTREZ_GENE_ID', 'SPOT_ID']
{'ID': ['1', '2', '3', '9', '10'], 'ENTREZ_GENE_ID': ['1', '2', '3', '9', '10'], 'SPOT_ID': [1.0, 2.0, 3.0, 9.0, 10.0]}

Searching for platform information in SOFT file:
Platform ID not found in first 100 lines

Searching for gene symbol information in SOFT file:


No explicit gene symbol references found in first 1000 lines

Checking for additional annotation files in the directory:
[]


### Step 6: Gene Identifier Mapping

In [7]:
# Looking at the annotation data, we can see it includes:
# ID: probe identifiers that match gene_data index
# ENTREZ_GENE_ID: Entrez Gene IDs which can serve as gene identifiers

# 1. Identify the appropriate columns for mapping
# From the preview, we can see that ID column in annotation matches the index in gene_data
# ENTREZ_GENE_ID appears to be the closest to gene identifiers we have

# Since the ENTREZ_GENE_ID is numeric, we'll check if it can be mapped to gene symbols
# We'll use the gene_mapping function from the library with necessary columns
mapping_df = get_gene_mapping(gene_annotation, 'ID', 'ENTREZ_GENE_ID')

print("\nGene mapping dataframe preview:")
print(preview_df(mapping_df, n=5))

# 3. Apply the gene mapping to convert probe-level data to gene-level data
gene_data = apply_gene_mapping(gene_data, mapping_df)

print("\nGene expression data after mapping:")
print(f"Shape: {gene_data.shape}")
print(f"First 10 gene identifiers: {list(gene_data.index[:10])}")

# Save the processed gene data to the output file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene data saved to {out_gene_data_file}")


Gene mapping dataframe preview:
{'ID': ['1', '2', '3', '9', '10'], 'Gene': ['1', '2', '3', '9', '10']}



Gene expression data after mapping:
Shape: (0, 204)
First 10 gene identifiers: []
Gene data saved to ../../output/preprocess/Eczema/gene_data/GSE123088.csv
